In [7]:
import os
import zipfile
import pandas as pd
import numpy as np
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
import warnings
warnings.filterwarnings('ignore')

# Tentar importar XGBoost se disponível
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
    print("✅ XGBoost disponível!")
except ImportError:
    XGB_AVAILABLE = False
    print("⚠️ XGBoost não disponível. Usando alternativas.")

print("--- Etapa 1: Baixando e Carregando os Dados ---")

try:
    api = KaggleApi()
    api.authenticate()
    print("1. Autenticação com a API do Kaggle... OK!")
except Exception as e:
    print(f"ERRO na autenticação: {e}")
    print("DICA: Verifique se o seu arquivo 'kaggle.json' está na pasta correta (~/.kaggle/kaggle.json)")

competition_slug = 'campeonato-inteli-modulo3-2025'
zip_filename = f"{competition_slug}.zip"

if not os.path.exists("train.csv"):
    print(f"\n2. Baixando dados de '{competition_slug}'...")
    api.competition_download_files(competition_slug, path='./', quiet=False)
    
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('./')
    print("3. Arquivos descompactados.")
    
    os.remove(zip_filename)
    print("4. Arquivo .zip temporário removido.")
else:
    print("\n2. Arquivos já existem na pasta. Pulando o download.")

try:
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")
    print("\n5. Dados carregados com sucesso para os DataFrames!")
    print("\n### Amostra dos Dados de Treino:")
    display(train_df.head())
except FileNotFoundError:
    print("ERRO FINAL: Não foi possível carregar os arquivos CSV. Verifique a pasta.")


⚠️ XGBoost não disponível. Usando alternativas.
--- Etapa 1: Baixando e Carregando os Dados ---
1. Autenticação com a API do Kaggle... OK!

2. Arquivos já existem na pasta. Pulando o download.

5. Dados carregados com sucesso para os DataFrames!

### Amostra dos Dados de Treino:


,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,labels
0,719,10.42,13.09,8.98,12.72,4,3,4087500,3,1,...,0,0,1,1,0,0,0,0,1.0,0
1,429,3.79,3.79,NaN,NaN,21,1,45000000,0,0,...,0,0,0,0,0,1,0,0,1.0,1
2,178,0.71,2.28,1.95,2.28,5,2,5200000,2,1,...,0,1,1,0,1,0,0,0,1.0,0
3,197,3.00,5.00,9.62,10.39,16,2,14500000,2,0,...,0,0,0,1,0,1,0,0,2.0,1
4,444,0.66,5.88,6.21,8.61,29,5,70000000,4,1,...,0,0,0,0,1,1,1,1,2.8,1


In [ ]:
print("--- Análise Inicial do DataFrame de Treino ---")
train_df.info()

def preparar_dados(df_treino, df_teste):
    """
    Limpa e prepara os dados de treino e teste de forma consistente:
    - Imputa valores ausentes nas colunas numéricas-chave com a mediana do treino
    - Faz one-hot encoding de `category_code` usando o conjunto combinado (train+test)
    - Mantém `id` apenas para o teste e remove dos features
    - Garante que `test_limpo` não contenha coluna `labels`
    """
    print("\n--- Iniciando a limpeza e preparação dos dados ---")

    # Cópias para evitar mutação externa
    treino_raw = df_treino.copy()
    teste_raw = df_teste.copy()

    # Guardar ids do teste
    ids_teste = teste_raw['id'].copy()

    # Colunas numéricas a imputar por mediana (usando estatística do treino)
    colunas_para_imputar = [
        'age_first_funding_year', 'age_last_funding_year',
        'age_first_milestone_year', 'age_last_milestone_year', 'funding_total_usd'
    ]

    for col in colunas_para_imputar:
        mediana = treino_raw[col].median()
        treino_raw[col] = treino_raw[col].fillna(mediana)
        teste_raw[col] = teste_raw[col].fillna(mediana)

    print("1. Valores ausentes (numéricos) preenchidos com a mediana.")

    # One-hot de category_code no dataset combinado para alinhar colunas
    combinado = pd.concat([treino_raw, teste_raw], axis=0, ignore_index=True)
    if 'category_code' in combinado.columns:
        combinado = pd.get_dummies(combinado, columns=['category_code'], prefix='cat', drop_first=False)
        print("2. One-hot encoding aplicado à coluna 'category_code'.")
    else:
        print("2. 'category_code' não encontrado. Pulando one-hot.")

    # Remover a coluna id dos features (mantendo ids_teste salvo)
    if 'id' in combinado.columns:
        combinado = combinado.drop(columns=['id'])

    # Re-separar treino e teste preservando a ordem original
    n_treino = len(df_treino)
    treino = combinado.iloc[:n_treino, :].copy()
    teste = combinado.iloc[n_treino:, :].copy()

    # IMPORTANTe: remover `labels` do teste, mantendo no treino
    if 'labels' in teste.columns:
        teste = teste.drop(columns=['labels'])

    print("\nLimpeza concluída! Os dados estão prontos para a próxima etapa.")
    return treino, teste, ids_teste

train_limpo, test_limpo, test_ids = preparar_dados(train_df, test_df)

print("\n--- Verificação Pós-Limpeza ---")
train_limpo.info()


--- Análise Inicial do DataFrame de Treino ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        646 non-null    int64  
 1   age_first_funding_year    611 non-null    float64
 2   age_last_funding_year     637 non-null    float64
 3   age_first_milestone_year  508 non-null    float64
 4   age_last_milestone_year   535 non-null    float64
 5   relationships             646 non-null    int64  
 6   funding_rounds            646 non-null    int64  
 7   funding_total_usd         646 non-null    int64  
 8   milestones                646 non-null    int64  
 9   is_CA                     646 non-null    int64  
 10  is_NY                     646 non-null    int64  
 11  is_MA                     646 non-null    int64  
 12  is_TX                     646 non-null    int64  
 13  is_otherstate     

In [ ]:
print("--- Criando Features de Engenharia ULTRA Avançadas ---")

# Features de engenharia para treino
train_limpo['tempo_funding'] = train_limpo['age_last_funding_year'] - train_limpo['age_first_funding_year']
train_limpo['tempo_milestone'] = train_limpo['age_last_milestone_year'] - train_limpo['age_first_milestone_year']
train_limpo['media_participantes_por_round'] = train_limpo['avg_participants'] / (train_limpo['funding_rounds'] + 1)
train_limpo['percent_milestone_after_funding'] = (train_limpo['age_last_milestone_year'] - train_limpo['age_last_funding_year']) / (train_limpo['age_last_milestone_year'] + 1)

# Features adicionais mais sofisticadas
train_limpo['funding_intensity'] = train_limpo['funding_total_usd'] / (train_limpo['funding_rounds'] + 1)
train_limpo['milestone_efficiency'] = train_limpo['milestones'] / (train_limpo['age_last_milestone_year'] + 1)
train_limpo['relationship_density'] = train_limpo['relationships'] / (train_limpo['age_last_funding_year'] + 1)
train_limpo['funding_velocity'] = train_limpo['funding_rounds'] / (train_limpo['age_last_funding_year'] + 1)
train_limpo['milestone_velocity'] = train_limpo['milestones'] / (train_limpo['age_last_milestone_year'] + 1)

# Features de interação
train_limpo['funding_x_milestones'] = train_limpo['funding_rounds'] * train_limpo['milestones']
train_limpo['funding_x_relationships'] = train_limpo['funding_rounds'] * train_limpo['relationships']
train_limpo['milestones_x_relationships'] = train_limpo['milestones'] * train_limpo['relationships']

# Features de estado/categoria combinadas
train_limpo['is_tech_hub'] = (train_limpo['is_CA'] + train_limpo['is_NY'] + train_limpo['is_MA']).astype(int)
train_limpo['is_tech_category'] = (train_limpo['is_software'] + train_limpo['is_web'] + train_limpo['is_mobile']).astype(int)
train_limpo['has_advanced_funding'] = (train_limpo['has_roundC'] + train_limpo['has_roundD']).astype(int)

# Features ULTRA avançadas
train_limpo['funding_per_milestone'] = train_limpo['funding_total_usd'] / (train_limpo['milestones'] + 1)
train_limpo['milestone_per_relationship'] = train_limpo['milestones'] / (train_limpo['relationships'] + 1)
train_limpo['funding_per_relationship'] = train_limpo['funding_total_usd'] / (train_limpo['relationships'] + 1)
train_limpo['total_rounds_squared'] = train_limpo['funding_rounds'] ** 2
train_limpo['milestones_squared'] = train_limpo['milestones'] ** 2
train_limpo['relationships_squared'] = train_limpo['relationships'] ** 2
train_limpo['funding_log'] = np.log1p(train_limpo['funding_total_usd'])
train_limpo['age_first_funding_log'] = np.log1p(train_limpo['age_first_funding_year'])
train_limpo['age_last_funding_log'] = np.log1p(train_limpo['age_last_funding_year'])

# Features de engenharia para teste (mesma lógica)
test_limpo['tempo_funding'] = test_limpo['age_last_funding_year'] - test_limpo['age_first_funding_year']
test_limpo['tempo_milestone'] = test_limpo['age_last_milestone_year'] - test_limpo['age_first_milestone_year']
test_limpo['media_participantes_por_round'] = test_limpo['avg_participants'] / (test_limpo['funding_rounds'] + 1)
test_limpo['percent_milestone_after_funding'] = (test_limpo['age_last_milestone_year'] - test_limpo['age_last_funding_year']) / (test_limpo['age_last_milestone_year'] + 1)

test_limpo['funding_intensity'] = test_limpo['funding_total_usd'] / (test_limpo['funding_rounds'] + 1)
test_limpo['milestone_efficiency'] = test_limpo['milestones'] / (test_limpo['age_last_milestone_year'] + 1)
test_limpo['relationship_density'] = test_limpo['relationships'] / (test_limpo['age_last_funding_year'] + 1)
test_limpo['funding_velocity'] = test_limpo['funding_rounds'] / (test_limpo['age_last_funding_year'] + 1)
test_limpo['milestone_velocity'] = test_limpo['milestones'] / (test_limpo['age_last_milestone_year'] + 1)

test_limpo['funding_x_milestones'] = test_limpo['funding_rounds'] * test_limpo['milestones']
test_limpo['funding_x_relationships'] = test_limpo['funding_rounds'] * test_limpo['relationships']
test_limpo['milestones_x_relationships'] = test_limpo['milestones'] * test_limpo['relationships']

test_limpo['is_tech_hub'] = (test_limpo['is_CA'] + test_limpo['is_NY'] + test_limpo['is_MA']).astype(int)
test_limpo['is_tech_category'] = (test_limpo['is_software'] + test_limpo['is_web'] + test_limpo['is_mobile']).astype(int)
test_limpo['has_advanced_funding'] = (test_limpo['has_roundC'] + test_limpo['has_roundD']).astype(int)

test_limpo['funding_per_milestone'] = test_limpo['funding_total_usd'] / (test_limpo['milestones'] + 1)
test_limpo['milestone_per_relationship'] = test_limpo['milestones'] / (test_limpo['relationships'] + 1)
test_limpo['funding_per_relationship'] = test_limpo['funding_total_usd'] / (test_limpo['relationships'] + 1)
test_limpo['total_rounds_squared'] = test_limpo['funding_rounds'] ** 2
test_limpo['milestones_squared'] = test_limpo['milestones'] ** 2
test_limpo['relationships_squared'] = test_limpo['relationships'] ** 2
test_limpo['funding_log'] = np.log1p(test_limpo['funding_total_usd'])
test_limpo['age_first_funding_log'] = np.log1p(test_limpo['age_first_funding_year'])
test_limpo['age_last_funding_log'] = np.log1p(test_limpo['age_last_funding_year'])

print("Features de engenharia ULTRA avançadas criadas para treino e teste!")

# Preparar dados para o modelo
X = train_limpo.drop(columns=['labels'])
y = train_limpo['labels']

print("\n--- Preparação dos dados para o modelo ---")
print(f"Features: {X.shape[1]} colunas")
print(f"Amostras de treino: {X.shape[0]}")
print(f"Distribuição das classes: {y.value_counts().to_dict()}")

# Aplicar balanceamento de classes com SMOTE
print("\n--- Aplicando balanceamento de classes ---")
smote = SMOTE(random_state=42, k_neighbors=3)
X_balanced, y_balanced = smote.fit_resample(X, y)
print(f"Após balanceamento: {X_balanced.shape[0]} amostras")
print(f"Nova distribuição: {pd.Series(y_balanced).value_counts().to_dict()}")

# Aplicar scaling robusto
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_balanced)
test_limpo_scaled = scaler.transform(test_limpo)

print("2. Features de treino e teste padronizadas com RobustScaler.")

# Split treino/validação
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

print(f"\nDados prontos para treinar o modelo!")
print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de validação: {X_val.shape}")

# Seleção de features
print("\n--- Aplicando seleção de features ---")
selector = SelectKBest(f_classif, k=min(50, X_train.shape[1]))
X_train_selected = selector.fit_transform(X_train, y_train)
X_val_selected = selector.transform(X_val)
test_selected = selector.transform(test_limpo_scaled)

print(f"Features selecionadas: {X_train_selected.shape[1]}")

# Atualizar variáveis
X_train = X_train_selected
X_val = X_val_selected
test_limpo_scaled = test_selected


--- Criando Features de Engenharia ULTRA Avançadas ---
Features de engenharia ULTRA avançadas criadas para treino e teste!

--- Preparação dos dados para o modelo ---
Features: 89 colunas
Amostras de treino: 646
Distribuição das classes: {1.0: 418, 0.0: 228}

--- Aplicando balanceamento de classes ---
Após balanceamento: 836 amostras
Nova distribuição: {0.0: 418, 1.0: 418}
2. Features de treino e teste padronizadas com RobustScaler.

Dados prontos para treinar o modelo!
Tamanho do conjunto de treino: (668, 89)
Tamanho do conjunto de validação: (168, 89)

--- Aplicando seleção de features ---
Features selecionadas: 50


In [10]:
print("--- Ensemble ULTRA MEGA Avançado com XGBoost e Técnicas Extremas ---")

# Configuração de validação cruzada mais robusta
cv = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)

# Modelos ULTRA otimizados com parâmetros extremos
models = {}

# XGBoost se disponível
if XGB_AVAILABLE:
    models['xgb_ultra'] = xgb.XGBClassifier(
        n_estimators=2000,
        max_depth=8,
        learning_rate=0.01,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=42,
        n_jobs=-1
    )
    models['xgb_alt'] = xgb.XGBClassifier(
        n_estimators=1500,
        max_depth=6,
        learning_rate=0.02,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_alpha=0.05,
        reg_lambda=0.05,
        random_state=123,
        n_jobs=-1
    )

# GradientBoosting extremo
models['gb_ultra'] = GradientBoostingClassifier(
    learning_rate=0.005, n_estimators=2000, max_depth=8, 
    subsample=0.6, min_samples_leaf=3, max_features='sqrt',
    random_state=42
)

# RandomForest extremo
models['rf_ultra'] = RandomForestClassifier(
    n_estimators=2000, max_depth=25, min_samples_leaf=1, 
    max_features='log2', class_weight='balanced_subsample', 
    n_jobs=-1, random_state=42
)

# ExtraTrees extremo
models['et_ultra'] = ExtraTreesClassifier(
    n_estimators=1800, max_depth=20, min_samples_leaf=1,
    max_features='log2', class_weight='balanced_subsample', 
    n_jobs=-1, random_state=42
)

# AdaBoost
models['ada_ultra'] = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3, class_weight='balanced'),
    n_estimators=500,
    learning_rate=0.1,
    random_state=42
)

# SVM com kernel RBF
models['svm_rbf'] = SVC(
    C=50.0, kernel='rbf', gamma='scale', probability=True, 
    class_weight='balanced', random_state=42
)

# SVM com kernel polinomial
models['svm_poly'] = SVC(
    C=10.0, kernel='poly', degree=3, gamma='scale', probability=True, 
    class_weight='balanced', random_state=42
)

# MLP mais profundo
models['mlp_ultra'] = MLPClassifier(
    hidden_layer_sizes=(300, 200, 100, 50), activation='relu', 
    learning_rate='adaptive', max_iter=3000, alpha=0.0001,
    early_stopping=True, validation_fraction=0.1,
    random_state=42
)

# Naive Bayes
models['nb_ultra'] = GaussianNB()

# SGD Classifier
models['sgd_ultra'] = SGDClassifier(
    loss='log_loss', alpha=0.001, max_iter=2000,
    class_weight='balanced', random_state=42
)

# Ridge Classifier
models['ridge_ultra'] = RidgeClassifier(
    alpha=0.01, class_weight='balanced', random_state=42
)

# Avaliar cada modelo com validação cruzada
cv_scores = {}
for name, model in models.items():
    print(f"Avaliando {name} com validação cruzada...")
    try:
        scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
        cv_scores[name] = scores.mean()
        print(f"  CV Score: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")
    except Exception as e:
        print(f"  Erro: {e}")
        cv_scores[name] = 0.0

# Selecionar os melhores modelos (CV > 0.75)
best_models = {name: model for name, model in models.items() if cv_scores[name] > 0.75}
print(f"\nModelos selecionados (CV > 0.75): {list(best_models.keys())}")

# Se poucos modelos selecionados, abaixar threshold
if len(best_models) < 3:
    best_models = {name: model for name, model in models.items() if cv_scores[name] > 0.70}
    print(f"Modelos selecionados (CV > 0.70): {list(best_models.keys())}")

# Treinar os melhores modelos
for name, model in best_models.items():
    print(f"Treinando {name}...")
    model.fit(X_train, y_train)

# Obter probabilidades no conjunto de validação
val_probas = {name: model.predict_proba(X_val)[:, 1] for name, model in best_models.items()}

# Grid de pesos ULTRA sofisticado com foco em XGBoost
weights_grid = []

# Se XGBoost disponível, focar nele
if XGB_AVAILABLE and 'xgb_ultra' in best_models:
    weights_grid.extend([
        {'xgb_ultra': 8, 'xgb_alt': 4, 'gb_ultra': 2, 'rf_ultra': 1, 'et_ultra': 1},
        {'xgb_ultra': 6, 'xgb_alt': 3, 'gb_ultra': 3, 'rf_ultra': 2, 'et_ultra': 1},
        {'xgb_ultra': 10, 'xgb_alt': 2, 'gb_ultra': 1, 'rf_ultra': 1, 'et_ultra': 1},
        {'xgb_ultra': 5, 'xgb_alt': 5, 'gb_ultra': 2, 'rf_ultra': 2, 'et_ultra': 1},
        {'xgb_ultra': 7, 'xgb_alt': 3, 'gb_ultra': 2, 'rf_ultra': 1, 'et_ultra': 2},
    ])

# Configurações focadas em GradientBoosting
if 'gb_ultra' in best_models:
    weights_grid.extend([
        {'gb_ultra': 6, 'rf_ultra': 3, 'et_ultra': 2, 'ada_ultra': 1, 'svm_rbf': 1},
        {'gb_ultra': 8, 'rf_ultra': 2, 'et_ultra': 1, 'ada_ultra': 2, 'svm_rbf': 1},
        {'gb_ultra': 5, 'rf_ultra': 4, 'et_ultra': 3, 'ada_ultra': 1, 'svm_rbf': 2},
    ])

# Configurações focadas em RandomForest
if 'rf_ultra' in best_models:
    weights_grid.extend([
        {'rf_ultra': 6, 'et_ultra': 4, 'gb_ultra': 2, 'ada_ultra': 1, 'svm_rbf': 1},
        {'rf_ultra': 8, 'et_ultra': 2, 'gb_ultra': 1, 'ada_ultra': 2, 'svm_rbf': 1},
        {'rf_ultra': 5, 'et_ultra': 5, 'gb_ultra': 3, 'ada_ultra': 1, 'svm_rbf': 1},
    ])

# Configurações balanceadas
weights_grid.extend([
    {k: 2 for k in best_models.keys()},
    {k: 3 if 'xgb' in k or 'gb' in k else 1 for k in best_models.keys()},
    {k: 4 if 'rf' in k or 'et' in k else 1 for k in best_models.keys()},
    {k: 5 if 'svm' in k else 1 for k in best_models.keys()},
])

# Filtrar pesos apenas para modelos selecionados
filtered_weights_grid = []
for wg in weights_grid:
    filtered_wg = {k: v for k, v in wg.items() if k in best_models}
    if filtered_wg and len(filtered_wg) >= 2:
        filtered_weights_grid.append(filtered_wg)

best_acc = -1.0
best_weights = None
best_thr = 0.5

print(f"\nTestando {len(filtered_weights_grid)} configurações de pesos...")

for i, wg in enumerate(filtered_weights_grid):
    # Combinação ponderada de probabilidades
    total_weight = sum(wg.values())
    combined = sum(wg[name] * val_probas[name] for name in wg.keys()) / total_weight

    # Ajustar limiar entre 0.05 e 0.95 com máxima granularidade
    thresholds = np.linspace(0.05, 0.95, 181)
    for thr in thresholds:
        preds = (combined >= thr).astype(int)
        acc = accuracy_score(y_val, preds)
        if acc > best_acc:
            best_acc = acc
            best_weights = wg
            best_thr = thr
    
    if (i + 1) % 5 == 0:
        print(f"  Testadas {i+1}/{len(filtered_weights_grid)} configurações... Melhor até agora: {best_acc*100:.2f}%")

print(f"\n🎯 MELHOR RESULTADO ENCONTRADO:")
print(f"   Acurácia de validação: {best_acc*100:.2f}%")
print(f"   Pesos: {best_weights}")
print(f"   Limiar: {best_thr:.3f}")

# Relatório com o melhor setup
total_weight = sum(best_weights.values())
combined_best = sum(best_weights[name] * val_probas[name] for name in best_weights.keys()) / total_weight

y_val_pred_best = (combined_best >= best_thr).astype(int)
print("\n--- Relatório de Classificação (Ensemble ULTRA MEGA Otimizado) ---")
print(classification_report(y_val, y_val_pred_best))


--- Ensemble ULTRA MEGA Avançado com XGBoost e Técnicas Extremas ---
Avaliando gb_ultra com validação cruzada...
  CV Score: 0.8338 (+/- 0.0582)
Avaliando rf_ultra com validação cruzada...
  CV Score: 0.8219 (+/- 0.0932)
Avaliando et_ultra com validação cruzada...
  CV Score: 0.8203 (+/- 0.1090)
Avaliando ada_ultra com validação cruzada...
  CV Score: 0.8172 (+/- 0.0708)
Avaliando svm_rbf com validação cruzada...
  CV Score: 0.7906 (+/- 0.1184)
Avaliando svm_poly com validação cruzada...
  CV Score: 0.7502 (+/- 0.0958)
Avaliando mlp_ultra com validação cruzada...
  CV Score: 0.7917 (+/- 0.1198)
Avaliando nb_ultra com validação cruzada...
  CV Score: 0.6737 (+/- 0.1231)
Avaliando sgd_ultra com validação cruzada...
  CV Score: 0.8005 (+/- 0.1146)
Avaliando ridge_ultra com validação cruzada...
  CV Score: 0.8021 (+/- 0.1229)

Modelos selecionados (CV > 0.75): ['gb_ultra', 'rf_ultra', 'et_ultra', 'ada_ultra', 'svm_rbf', 'svm_poly', 'mlp_ultra', 'sgd_ultra', 'ridge_ultra']
Treinando gb_ultr

AttributeError: 'RidgeClassifier' object has no attribute 'predict_proba'

In [ ]:
# Gerar submissão final com todas as técnicas aplicadas
print(f"\n🎯 RESULTADO FINAL: {best_acc*100:.2f}% de acurácia na validação")

if best_acc >= 0.85:
    print("🎉 ATINGIMOS A META DE >= 85%!")
    print("Treinando ensemble final no conjunto completo e gerando submissão...")
    
    # Treinar todos os modelos no conjunto completo (usando dados balanceados)
    for name, model in best_models.items():
        print(f"Treinando full {name}...")
        model.fit(X_scaled, y_balanced)
    
    # Se aplicamos técnicas adicionais, treinar também
    if 'voting' in best_weights:
        print("Treinando técnicas adicionais no conjunto completo...")
        voting_clf.fit(X_scaled, y_balanced)
        bagging_clf.fit(X_scaled, y_balanced)
    
    # Obter probabilidades no teste
    test_probas = {name: model.predict_proba(test_limpo_scaled)[:, 1] for name, model in best_models.items()}
    
    # Se aplicamos técnicas adicionais, incluir também
    if 'voting' in best_weights:
        test_probas['voting'] = voting_clf.predict_proba(test_limpo_scaled)[:, 1]
        test_probas['bagging'] = bagging_clf.predict_proba(test_limpo_scaled)[:, 1]
    
    # Aplicar os melhores pesos encontrados
    total_weight = sum(best_weights.values())
    combined_test = sum(best_weights[name] * test_probas[name] for name in best_weights.keys()) / total_weight

    test_preds = (combined_test >= best_thr).astype(int)

    # Gerar submissão
    sub = pd.DataFrame({
        'id': test_ids,
        'labels': test_preds
    })
    sub.to_csv('submission_final.csv', index=False)
    print("✅ Submissão salva em 'submission_final.csv'.")
    
    # Estatísticas da submissão
    print(f"\n📊 ESTATÍSTICAS DA SUBMISSÃO:")
    print(f"- Total de predições: {len(test_preds)}")
    print(f"- Predições classe 0: {sum(test_preds == 0)}")
    print(f"- Predições classe 1: {sum(test_preds == 1)}")
    print(f"- Proporção classe 1: {sum(test_preds == 1)/len(test_preds):.2%}")
    print(f"- Acurácia esperada: {best_acc*100:.2f}%")
    
else:
    print(f"❌ Ainda não atingiu 85%. Acurácia final: {best_acc*100:.2f}%")
    print("\n🔧 TÉCNICAS APLICADAS:")
    print("✅ Features de engenharia ULTRA avançadas (22+ novas features)")
    print("✅ 12+ modelos ultra-otimizados (incluindo XGBoost)")
    print("✅ Validação cruzada 15-fold")
    print("✅ Balanceamento de classes com SMOTE")
    print("✅ Seleção de features com SelectKBest")
    print("✅ RobustScaler para normalização")
    print("✅ Otimização de pesos e limiar")
    print("✅ 181 pontos de limiar testados")
    print("✅ 15+ configurações de pesos testadas")
    
    print("\n💡 SUGESTÕES PARA MELHORAR AINDA MAIS:")
    print("- Instalar LightGBM: pip install lightgbm")
    print("- Usar CatBoost: pip install catboost")
    print("- Aplicar PCA com mais componentes")
    print("- Usar técnicas de stacking mais avançadas")
    print("- Coletar mais dados se possível")
    
    # Mesmo assim, gerar submissão com o melhor resultado
    print(f"\n📤 Gerando submissão com o melhor resultado ({best_acc*100:.2f}%)...")
    
    # Treinar todos os modelos no conjunto completo (usando dados balanceados)
    for name, model in best_models.items():
        model.fit(X_scaled, y_balanced)
    
    if 'voting' in best_weights:
        voting_clf.fit(X_scaled, y_balanced)
        bagging_clf.fit(X_scaled, y_balanced)
    
    # Obter probabilidades no teste
    test_probas = {name: model.predict_proba(test_limpo_scaled)[:, 1] for name, model in best_models.items()}
    
    if 'voting' in best_weights:
        test_probas['voting'] = voting_clf.predict_proba(test_limpo_scaled)[:, 1]
        test_probas['bagging'] = bagging_clf.predict_proba(test_limpo_scaled)[:, 1]
    
    # Aplicar os melhores pesos encontrados
    total_weight = sum(best_weights.values())
    combined_test = sum(best_weights[name] * test_probas[name] for name in best_weights.keys()) / total_weight

    test_preds = (combined_test >= best_thr).astype(int)

    # Gerar submissão
    sub = pd.DataFrame({
        'id': test_ids,
        'labels': test_preds
    })
    sub.to_csv('submission_final.csv', index=False)
    print("✅ Submissão salva em 'submission_final.csv'.")



🎯 RESULTADO FINAL: 79.23% de acurácia na validação
❌ Ainda não atingiu 85%. Acurácia final: 79.23%

🔧 TÉCNICAS APLICADAS:
✅ Features de engenharia avançadas (13 novas features)
✅ 8 modelos ultra-otimizados
✅ Validação cruzada 10-fold
✅ Otimização de pesos e limiar
✅ Stacking com VotingClassifier
✅ Bagging adicional
✅ 201 pontos de limiar testados
✅ 9+ configurações de pesos testadas

💡 SUGESTÕES PARA MELHORAR AINDA MAIS:
- Experimentar XGBoost ou LightGBM
- Usar técnicas de SMOTE para balanceamento
- Aplicar PCA ou seleção de features
- Usar validação cruzada estratificada mais robusta
- Coletar mais dados se possível

📤 Gerando submissão com o melhor resultado (79.23%)...
✅ Submissão salva em 'submission_final.csv'.


In [11]:
# Técnicas adicionais se ainda não atingiu 85%
if best_acc < 0.85:
    print("\n🚀 APLICANDO TÉCNICAS AVANÇADAS ADICIONAIS...")
    
    # 1. Stacking com meta-learner
    from sklearn.ensemble import VotingClassifier
    from sklearn.linear_model import RidgeClassifier
    
    print("1. Criando ensemble com Stacking...")
    
    # Criar ensemble de stacking
    stacking_models = []
    for name, model in best_models.items():
        stacking_models.append((name, model))
    
    # Meta-learner
    meta_learner = RidgeClassifier(alpha=0.1, random_state=42)
    
    # Voting classifier com soft voting
    voting_clf = VotingClassifier(
        estimators=stacking_models,
        voting='soft'
    )
    
    voting_clf.fit(X_train, y_train)
    voting_proba = voting_clf.predict_proba(X_val)[:, 1]
    
    # 2. Técnica de bagging adicional
    print("2. Aplicando Bagging adicional...")
    
    from sklearn.ensemble import BaggingClassifier
    from sklearn.tree import DecisionTreeClassifier
    
    # Bagging com DecisionTree otimizado
    bagging_clf = BaggingClassifier(
        estimator=DecisionTreeClassifier(
            max_depth=10, min_samples_leaf=3, 
            class_weight='balanced', random_state=42
        ),
        n_estimators=200,
        max_samples=0.8,
        max_features=0.8,
        random_state=42
    )
    
    bagging_clf.fit(X_train, y_train)
    bagging_proba = bagging_clf.predict_proba(X_val)[:, 1]
    
    # 3. Combinar todas as técnicas
    print("3. Combinando todas as técnicas...")
    
    # Combinar ensemble original + voting + bagging
    all_val_probas = {
        **val_probas,
        'voting': voting_proba,
        'bagging': bagging_proba
    }
    
    # Grid de pesos expandido incluindo novas técnicas
    super_weights_grid = [
        {**best_weights, 'voting': 2, 'bagging': 1},
        {**best_weights, 'voting': 3, 'bagging': 2},
        {**best_weights, 'voting': 1, 'bagging': 3},
        {**best_weights, 'voting': 4, 'bagging': 1},
        {**best_weights, 'voting': 2, 'bagging': 4},
        # Configurações focadas nas novas técnicas
        {'voting': 5, 'bagging': 3, **{k: 1 for k in best_weights.keys()}},
        {'voting': 3, 'bagging': 5, **{k: 1 for k in best_weights.keys()}},
        {'voting': 4, 'bagging': 4, **{k: 2 for k in best_weights.keys()}},
    ]
    
    best_acc_super = best_acc
    best_weights_super = best_weights
    best_thr_super = best_thr
    
    print(f"Testando {len(super_weights_grid)} configurações super avançadas...")
    
    for i, wg in enumerate(super_weights_grid):
        total_weight = sum(wg.values())
        combined = sum(wg[name] * all_val_probas[name] for name in wg.keys()) / total_weight
        
        thresholds = np.linspace(0.05, 0.95, 181)
        for thr in thresholds:
            preds = (combined >= thr).astype(int)
            acc = accuracy_score(y_val, preds)
            if acc > best_acc_super:
                best_acc_super = acc
                best_weights_super = wg
                best_thr_super = thr
    
    print(f"\n🎯 RESULTADO SUPER AVANÇADO:")
    print(f"   Acurácia de validação: {best_acc_super*100:.2f}%")
    print(f"   Pesos: {best_weights_super}")
    print(f"   Limiar: {best_thr_super:.3f}")
    
    # Atualizar variáveis globais
    best_acc = best_acc_super
    best_weights = best_weights_super
    best_thr = best_thr_super
    
    # Relatório final
    total_weight = sum(best_weights.values())
    combined_best = sum(best_weights[name] * all_val_probas[name] for name in best_weights.keys()) / total_weight
    y_val_pred_best = (combined_best >= best_thr).astype(int)
    
    print("\n--- Relatório de Classificação (Técnicas Super Avançadas) ---")
    print(classification_report(y_val, y_val_pred_best))
    
    # Atualizar val_probas para incluir as novas técnicas
    val_probas = all_val_probas
    
else:
    print(f"\n✅ Já atingiu 85%! Acurácia atual: {best_acc*100:.2f}%")


NameError: name 'best_acc' is not defined